# Elo Calculations Notebook

In this notebook, we will devise a way to quickly compute the most up-to-date elo rating for a given team.

Additionally, we will need to find a way to find to find the nearest future games so we may predict the winner. 

In [1]:
# Import statements
import pandas as pd
from datetime import datetime
from nba_api.stats.static import teams

from nba_api.stats.endpoints import teamgamelogs
from nba_api.stats.endpoints import teaminfocommon

In [2]:
teamGameLogs = teamgamelogs.TeamGameLogs(season_nullable='2023-24')
df = teamGameLogs.get_data_frames()[0]

df

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG
0,2023-24,1610612750,MIN,Minnesota Timberwolves,0022301108,2024-04-03T00:00:00,MIN vs. TOR,W,48.0,49,...,55,182,53,136,1344,63,1794,152,8,1
1,2023-24,1610612761,TOR,Toronto Raptors,0022301108,2024-04-03T00:00:00,TOR @ MIN,L,48.0,33,...,2121,2182,1940,644,2029,331,2158,2256,2269,1
2,2023-24,1610612763,MEM,Memphis Grizzlies,0022301107,2024-04-03T00:00:00,MEM @ MIL,W,48.0,46,...,1314,182,1714,249,2142,63,1946,1322,549,1
3,2023-24,1610612766,CHA,Charlotte Hornets,0022301102,2024-04-03T00:00:00,CHA vs. POR,L,48.0,32,...,1791,492,306,249,130,331,965,2250,1236,1
4,2023-24,1610612753,ORL,Orlando Magic,0022301109,2024-04-03T00:00:00,ORL @ NOP,W,48.0,41,...,957,935,497,1936,24,1313,84,909,595,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,2023-24,1610612751,BKN,Brooklyn Nets,0022300067,2023-10-25T00:00:00,BKN vs. CLE,L,48.0,42,...,498,1933,1380,644,1344,484,292,1159,1139,1
2272,2023-24,1610612744,GSW,Golden State Warriors,0022300062,2023-10-24T00:00:00,GSW vs. PHX,L,48.0,36,...,2121,492,180,644,1634,1864,414,1765,1294,1
2273,2023-24,1610612756,PHX,Phoenix Suns,0022300062,2023-10-24T00:00:00,PHX @ GSW,W,48.0,42,...,1647,2045,1714,420,1344,1713,292,1521,930,1
2274,2023-24,1610612747,LAL,Los Angeles Lakers,0022300061,2023-10-24T00:00:00,LAL @ DEN,L,48.0,41,...,1647,715,1714,1272,1344,867,1794,1576,1776,1


In [3]:
# Set initial Elo ratings manually

initial_elo = dict()  # Create a dictionary to hold initial elos

# ELO RATINGS AT START OF 2023-24 SEASON, from https://neilpaine.substack.com/p/2023-24-nba-elo-ratings
initial_elo["ATL"] = 1500
initial_elo["BKN"] = 1431
initial_elo["BOS"] = 1771
initial_elo["CHA"] = 1291
initial_elo["CHI"] = 1471
initial_elo["CLE"] = 1556
initial_elo["DAL"] = 1636
initial_elo["DEN"] = 1650
initial_elo["DET"] = 1287
initial_elo["GSW"] = 1603
initial_elo["HOU"] = 1531
initial_elo["IND"] = 1583
initial_elo["LAC"] = 1543
initial_elo["LAL"] = 1571
initial_elo["MEM"] = 1326
initial_elo["MIA"] = 1555
initial_elo["MIL"] = 1584
initial_elo["MIN"] = 1674
initial_elo["NOP"] = 1606
initial_elo["NYK"] = 1569
initial_elo["OKC"] = 1631
initial_elo["ORL"] = 1529
initial_elo["PHI"] = 1483
initial_elo["PHX"] = 1613
initial_elo["POR"] = 1300
initial_elo["SAC"] = 1546
initial_elo["SAS"] = 1372
initial_elo["TOR"] = 1291
initial_elo["UTA"] = 1365
initial_elo["WAS"] = 1282